In [1]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.distance import geodesic
import pandas as pd
import numpy as np
import random
import time
import matplotlib.pyplot as plt

# Initialize geolocator with increased timeout
geolocator = Nominatim(user_agent="tsp-geolocator", timeout=10)

# List of major Indian cities
cities = [
    "Delhi", "Mumbai", "Bangalore", "Hyderabad", "Kolkata", "Chennai", "Pune", "Ahmedabad",
    "Jaipur", "Lucknow", "Surat", "Kanpur", "Nagpur", "Visakhapatnam", "Bhopal", "Patna",
    "Vadodara", "Agra", "Nashik", "Vijayawada", "Coimbatore", "Indore", "Thane", "Ludhiana",
    "Madurai", "Bhubaneswar", "Amritsar", "Varanasi", "Raipur", "Aurangabad", "Ranchi",
    "Jodhpur", "Guwahati", "Chandigarh", "Mysore", "Gwalior", "Noida", "Faridabad", "Ghaziabad",
    "Meerut", "Rajkot", "Dhanbad", "Jalandhar", "Kota", "Bareilly", "Bikaner", "Agartala",
    "Jammu", "Udaipur", "Dehradun", "Allahabad", "Srinagar", "Mangalore"
]

# Dictionary to store city coordinates
city_coordinates = {}

# Function to get coordinates with retry logic
def get_coordinates(city, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(city + ", India")
            if location:
                return (location.latitude, location.longitude)
            else:
                print(f"Coordinates for {city} not found.")
                return None
        except GeocoderTimedOut:
            print(f"Geocoder timed out for {city}, attempt {attempt + 1}/{retries}")
            time.sleep(2)
    return None

# Get coordinates for each city
for city in cities:
    coordinates = get_coordinates(city)
    if coordinates:
        city_coordinates[city] = coordinates

# Convert to DataFrame
city_df = pd.DataFrame.from_dict(city_coordinates, orient='index', columns=['Latitude', 'Longitude'])
city_df.to_csv('indian_cities_coordinates.csv', index_label='City')

# Calculate distance matrix
def calculate_distance_matrix(city_df):
    cities = city_df.index
    distance_matrix = pd.DataFrame(index=cities, columns=cities)
    for city1 in cities:
        for city2 in cities:
            if city1 == city2:
                distance_matrix.loc[city1, city2] = 0
            else:
                coord1 = (city_df.loc[city1, 'Latitude'], city_df.loc[city1, 'Longitude'])
                coord2 = (city_df.loc[city2, 'Latitude'], city_df.loc[city2, 'Longitude'])
                distance_matrix.loc[city1, city2] = geodesic(coord1, coord2).kilometers
    return distance_matrix

distance_matrix = calculate_distance_matrix(city_df)
distance_matrix.to_csv('distance_matrix.csv')
distance_array = distance_matrix.values

# Initial population creation
def create_initial_population(population_size, city_names):
    population = []
    for _ in range(population_size):
        route_indices = np.random.permutation(len(city_names)).tolist()
        route = [(city_names[i], i) for i in route_indices]
        population.append(route)
    return population

population_size = 20
cities = distance_matrix.columns.tolist()
initial_population = create_initial_population(population_size, cities)

# Fitness calculation
def calculate_route_distance(route, distance_matrix):
    total_distance = 0
    num_cities = len(route)
    for i in range(num_cities):
        current_city_index = route[i][1]
        next_city_index = route[(i + 1) % num_cities][1]
        total_distance += distance_matrix[current_city_index, next_city_index]
    return total_distance

def calculate_fitness(route, distance_matrix):
    total_distance = calculate_route_distance(route, distance_matrix)
    return 1 / total_distance if total_distance > 0 else float('inf')

# Roulette Wheel Selection
def roulette_wheel_selection(population, fitness_values, num_parents):
    total_fitness = sum(fitness_values)
    selection_probs = np.array(fitness_values) / total_fitness
    selection_probs = selection_probs / selection_probs.sum()
    parents_indices = np.random.choice(len(population), size=num_parents, p=selection_probs)
    return [population[i] for i in parents_indices]

# Crossover function (Order Crossover - OX)
def order_crossover(parent1, parent2):
    length = len(parent1)
    start, end = sorted(random.sample(range(length), 2))
    offspring = [None] * length
    offspring[start:end + 1] = parent1[start:end + 1]
    pos = (end + 1) % length
    for city in parent2:
        if city not in offspring:
            offspring[pos] = city
            pos = (pos + 1) % length
    return offspring

def crossover_population(parents, crossover_prob=0.6):
    offspring = []
    num_parents = len(parents)
    for i in range(0, num_parents - 1, 2):
        parent1, parent2 = parents[i], parents[i + 1]
        if random.random() < crossover_prob:
            child1 = order_crossover(parent1, parent2)
            child2 = order_crossover(parent2, parent1)
            offspring.extend([child1, child2])
        else:
            offspring.extend([parent1, parent2])
    if num_parents % 2 != 0:
        offspring.append(parents[-1])
    return offspring

# Mutation function
def mutate_route(route, mutation_prob=0.1):
    mutated_route = route[:]
    if random.random() < mutation_prob:
        idx1, idx2 = random.sample(range(len(route)), 2)
        mutated_route[idx1], mutated_route[idx2] = mutated_route[idx2], mutated_route[idx1]
    return mutated_route

def mutate_population(population, mutation_prob=0.1):
    return [mutate_route(route, mutation_prob) for route in population]

# Main Genetic Algorithm loop
generations = 100
num_parents = population_size // 2

for generation in range(generations):
    fitness_values = [calculate_fitness(route, distance_array) for route in initial_population]
    selected_parents = roulette_wheel_selection(initial_population, fitness_values, num_parents)
    offspring_population = crossover_population(selected_parents)
    mutated_population = mutate_population(offspring_population, mutation_prob=0.2)
    
    # Merge populations and retain the best routes (elitism)
    total_population = initial_population + mutated_population
    fitness_values = [calculate_fitness(route, distance_array) for route in total_population]
    sorted_population = [route for _, route in sorted(zip(fitness_values, total_population), key=lambda x: x[0], reverse=True)]
    initial_population = sorted_population[:population_size]
    
    # Output the best route every 10 generations
    if generation % 10 == 0 or generation == generations - 1:
        best_route = initial_population[0]
        best_distance = 1 / calculate_fitness(best_route, distance_array)
        print(f"Generation {generation+1}: Best route distance = {best_distance:.2f}")

print("Final best route:", initial_population[0])
print("Final best route distance:", 1 / calculate_fitness(initial_population[0], distance_array))


Generation 1: Best route distance = 46710.40
Generation 11: Best route distance = 39258.59
Generation 21: Best route distance = 38502.53
Generation 31: Best route distance = 37630.32
Generation 41: Best route distance = 35930.92
Generation 51: Best route distance = 35059.33
Generation 61: Best route distance = 33992.21
Generation 71: Best route distance = 31866.21
Generation 81: Best route distance = 31626.85
Generation 91: Best route distance = 30577.10
Generation 100: Best route distance = 29302.62
Final best route: [('Kota', 43), ('Udaipur', 48), ('Indore', 21), ('Nagpur', 12), ('Aurangabad', 29), ('Ranchi', 30), ('Bhubaneswar', 25), ('Kanpur', 11), ('Bhopal', 14), ('Bikaner', 45), ('Agra', 17), ('Amritsar', 26), ('Ludhiana', 23), ('Srinagar', 51), ('Bareilly', 44), ('Raipur', 28), ('Varanasi', 27), ('Jalandhar', 42), ('Jodhpur', 31), ('Chandigarh', 33), ('Faridabad', 37), ('Dehradun', 49), ('Vadodara', 16), ('Mumbai', 1), ('Ahmedabad', 7), ('Allahabad', 50), ('Gwalior', 35), ('Patn